<a href="https://colab.research.google.com/github/mathenaangeles/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

Sentiment analysis or opinion mining is a **Natural Language Processing (NLP)** technique used to determine if the emotional tone of a given text is positive or negative. NLP is a branch of **Artificial Intelligence (AI)** that enables computers to comprehend, generate, and manipulate human languages.

## 1. Installation

[Natural Language Toolkit (NLTK)](https://www.nltk.org/)

In [ ]:
import nltk
nltk.download('twitter_samples')
nltk.download('wordnet') # English lexical database
!unzip -o '/root/nltk_data/corpora/wordnet.zip' -d '/root/nltk_data/corpora/'
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Archive:  /root/nltk_data/corpora/wordnet.zip
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wordnet/noun.exc  
  inflating: /root/nltk_data/corpora/wordnet/verb.exc  
  inflating: /root/nltk_data/corpora/wordnet/README  
  inflating: /root/nltk_data/corpora/wordnet/index.sense  
  inflating: /root/nltk_data/corpora/wordnet/data.noun  
  inflating: /root/nltk_data/corpora/wordnet/data.adv  
  inflating: /root/nltk_data/corpora/wordnet/i

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.corpus import twitter_samples
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
tweets = twitter_samples.strings('tweets.20150430-223406.json')

## 2. Tokenization and Lemmatization
**Tokenization** involes spliting the text into smaller units called *tokens* that can be more easily assigned meaning. **Lemmatization** is a normalization technique that transforms a word into its root form.

In [ ]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

In [ ]:
print(positive_tweet_tokens[5])

['@BhaktisBanter', '@PallaviRuhail', 'This', 'one', 'is', 'irresistible', ':)', '#FlipkartFashionFriday', 'http://t.co/EbZ0L2VENM']


In [ ]:
from nltk.tag import pos_tag # part-of-speech tagging
from nltk.stem.wordnet import WordNetLemmatizer
import re, string

def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        # Remove hyperlinks, Twitter handles, punctuations, and speacial charaters.
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub('(@[A-Za-z0-9_]+)','', token)
        if tag.startswith('NN'): # noun
            pos = 'n'
        elif tag.startswith('VB'): # verb
            pos = 'v'
        elif tag.startswith('RB'): # adverb
            pos = 'r'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        # Remove stop words.
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [ ]:
print(positive_cleaned_tokens_list[5])

['one', 'irresistible', ':)', '#flipkartfashionfriday']


In [ ]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [ ]:
from nltk import FreqDist

all_pos_words = get_all_words(positive_cleaned_tokens_list)
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

all_neg_words = get_all_words(negative_cleaned_tokens_list)
freq_dist_neg = FreqDist(all_neg_words)
print(freq_dist_neg.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('get', 263), ('good', 261), ('thank', 253)]
[(':(', 4585), (':-(', 501), ("i'm", 343), ('...', 332), ('get', 325), ('miss', 291), ('go', 275), ('please', 275), ('want', 246), ('like', 218)]


## 3. Training and Testing

A Naive Bayes classifier is a probabilistic machine learning model that is based on applying [**Bayes' theorem**](https://en.wikipedia.org/wiki/Bayes%27_theorem) with an assumption of independence among predictors.

In [ ]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [ ]:
import random

positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]
dataset = positive_dataset + negative_dataset
random.shuffle(dataset)

train_data = dataset[:7000] # 70% of the data is for training
test_data = dataset[7000:] # 30% of the data is for testing

In [ ]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

In [ ]:
print("Accuracy:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy: 0.995
Most Informative Features
                      :) = True           Positi : Negati =    995.7 : 1.0
                follower = True           Positi : Negati =     33.9 : 1.0
                    glad = True           Positi : Negati =     23.3 : 1.0
                     sad = True           Negati : Positi =     23.0 : 1.0
                     bam = True           Positi : Negati =     22.0 : 1.0
                     x15 = True           Negati : Positi =     15.2 : 1.0
              appreciate = True           Positi : Negati =     14.1 : 1.0
               community = True           Positi : Negati =     14.1 : 1.0
                    damn = True           Negati : Positi =     13.9 : 1.0
                 welcome = True           Positi : Negati =     13.4 : 1.0
None


In [ ]:
from nltk.tokenize import word_tokenize

sample_tweet = "No more girlbossing I want to girl rest girl sleep girl lay down"
sample_tokens = remove_noise(word_tokenize(sample_tweet))
print(classifier.classify(dict([token, True] for token in sample_tokens)))

Negative
